# SSAM Spatial Transcriptomics
Author: **Gabriel Emilio Herrera Oropeza** <br/>
Date: 07/10/2021

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ssam
from matplotlib.path import Path
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib_scalebar.scalebar import ScaleBar
from tqdm import tqdm
import random
from matplotlib.colors import ListedColormap

### Read Data

In [ ]:
data = pd.read_csv("/Volumes/emilio_passport/wellcome/rotations/spagnoli/data/reads_Sample1a_LowThreshold_CARTANA2002901_20190702.csv")

### Transform Data
Because SSAM analysis is rooted in a cellular scale we transform the coordinates from a laboratory system into micrometers.

In [ ]:
um_per_pixel = 0.165

In [ ]:
data.X = (data.X - data.X.min()) * um_per_pixel + 10
data.Y = (data.Y - data.Y.min()) * um_per_pixel + 10

### Prepare data

In [ ]:
width = data.X.max() - data.X.min() + 10
height = data.Y.max() - data.Y.min() + 10

In [ ]:
grouped = data.groupby('gene').agg(list)
genes = list(grouped.index)

In [ ]:
coord_list = []
for target, coords in tqdm(grouped.iterrows(), total = grouped.shape[0]):
    coord_list.append(np.array(list(zip(*coords))))

### Create the SSAMDataset object

In [ ]:
ds = ssam.SSAMDataset(genes, coord_list, width, height)

### Create vector field

In [ ]:
analysis = ssam.SSAMAnalysis(
    ds,
    ncores = 10, # used for kde step
    save_dir = "../data/ssam_approach/",
    verbose = True
)

In [ ]:
analysis.run_kde(bandwidth = 2.5, use_mmap = False)

### Create Mask

In [ ]:
# Define coordinate of area of interest
low_xlim = 1600
high_xlim = 2300
low_ylim = 2600
high_ylim = 1900

In [ ]:
# Manual area annotation
xy = np.array([[low_xlim, high_ylim],
               [high_xlim, high_ylim],
               [high_xlim, low_ylim],
               [low_xlim, low_ylim]])

# Extract coordinates from SSAMDataset
x, y = np.meshgrid(np.arange(ds.vf.shape[0]), np.arange(ds.vf.shape[1]))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T

path = Path(xy)
input_mask = path.contains_points(points)
input_mask = input_mask.reshape((ds.vf.shape[1], ds.vf.shape[0], 1)).swapaxes(0, 1)
output_mask = input_mask

In [ ]:
# Show figure with area of interest
patch = Polygon(xy, True)
p = PatchCollection([patch], alpha=0.4)

plt.figure(figsize=[8, 8])
ds.plot_l1norm(cmap="Greys",rotate = 1)
plt.gca().add_collection(p)
plt.gca().invert_xaxis() 
plt.axis('off')
plt.tight_layout()
plt.savefig('../figures/ssam_sample1a_regionInt.pdf', dpi = 500)
plt.show()

### Find local maxima

In [ ]:
analysis.find_localmax(
    search_size=3,
    #min_norm=0.2, # the total gene expression threshold
    #min_expression=0.2, # the per gene expression threshold
    mask=input_mask, 
    )

In [ ]:
# Plot local maxima 
plt.figure(figsize=[8, 8])
ds.plot_l1norm(cmap="Greys",rotate = 1)
ds.plot_localmax(c="Blue", s=0.1, rotate = 1)

patch = Polygon(xy, facecolor="black", edgecolor="red", linewidth=10, ls="-")
p = PatchCollection([patch], alpha=0.4)
plt.gca().add_collection(p)

scalebar = ScaleBar(1, 'um') # 1 pixel = 1um
plt.gca().add_artist(scalebar)
plt.gca().invert_xaxis() 
plt.tight_layout()
plt.axis('off')
plt.savefig('../figures/ssam_sample1a_localMax.pdf', dpi = 500)
plt.show()

### Normalise vectors

In [ ]:
analysis.normalize_vectors_sctransform()

### Find clusters

In [ ]:
# Clustering uses the Louvain method
analysis.cluster_vectors(
    min_cluster_size = 0,
    pca_dims = 11,
    resolution = 0.6,
    metric = 'correlation')

In [ ]:
# Function for generating random colours for clusters
def colors_(n):
    colors = []
    for j in range(n):
        rand_color = "#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])
        colors.append(rand_color)
    return colors

In [ ]:
# Generate a random color for each cluster
colors_list = colors_(n = 23)

In [ ]:
# Plot tSNE with clusters
plt.figure(figsize=[10, 8])
ds.plot_tsne(pca_dims = 11, metric = "correlation", s = 5, run_tsne = True, colors = colors_list)
dct_legend = {}
for n, c in enumerate(colors_list):
    dct_legend[n] = plt.scatter([], [], color = c)

plt.title("tSNE - Louvain Clustering")
plt.legend([v for k, v in dct_legend.items()], [str(int(k)) for k, v in dct_legend.items()], bbox_to_anchor = (1.02, 1), loc = 'upper left')
plt.tight_layout()
plt.savefig('../figures/ssam_sample1a_tSNE.pdf', dpi = 500)
plt.show()

In [ ]:
# Plot UMAP with clusters
plt.figure(figsize=[10, 8])
ds.plot_umap(run_umap = False,
             pca_dims = 11, 
             metric = 'correlation',
             exclude_bad_clusters = True,
             s = 5,
             colors = colors_list
         )
dct_legend = {}
for n, c in enumerate(colors_list):
    dct_legend[n] = plt.scatter([], [], color = c)
    
plt.title("UMAP - Louvain Clustering")
plt.legend([v for k, v in dct_legend.items()], [str(int(k)) for k, v in dct_legend.items()], bbox_to_anchor = (1.02, 1), loc = 'upper left')
plt.tight_layout()
plt.savefig('../figures/ssam_sample1a_UMAP.pdf', dpi = 500)
plt.show()

### Cell types identification

In [ ]:
# Map cell types using clustering
analysis.map_celltypes()

filter_params = {
    "block_size": 151,
    "method": "mean",
    "mode": "constant",
    "offset": 0.2
    }

analysis.filter_celltypemaps(min_norm = 0.01, 
                             #filter_params=filter_params, 
                             min_r=0.6, 
                             fill_blobs=True, 
                             min_blob_area=10, 
                             output_mask=output_mask
                            )

In [ ]:
# Plot cell types according to clusters
plt.figure(figsize=[8, 8])
ds.plot_celltypes_map(rotate=1, set_alpha=False, colors = colors_list)
plt.axis('off')
plt.gca().invert_xaxis()
plt.ylim(2600, 1900)
plt.xlim(1600, 2300)
plt.title("In Situ Clusters")
plt.tight_layout()
plt.savefig('../figures/ssam_sample1a_insituClusters.pdf', dpi = 500)
plt.show()

### Plot modified-diagnostic plot
As the original function only allowed to do this plot when scRNAseq data was available, I modified the function so it could be shown just for spatial transcriptomics data.

In [ ]:
for n, c in enumerate(ds.centroids):
    plt.figure(figsize=[50, 15])
    ds.plot_mod_diagnostic_plot(n,
                                low_ylim,
                                high_ylim,  
                                low_xlim, 
                                high_xlim,
                                cluster_color=colors_list[n], 
                                use_embedding="tsne", 
                                rotate = 1
                               )
    plt.tight_layout()
    plt.savefig(f'../figures/ssam_sample1a_diagnosticPlot_c{n}.pdf', dpi = 500)
    plt.show()

### Find tissue domains

In [ ]:
analysis.bin_celltypemaps(step = 10, radius = 100)

In [ ]:
analysis.find_domains(#n_clusters=20, 
    merge_remote=True, 
    merge_thres=0.7, 
    norm_thres=1500
)

In [ ]:
# Plot domains individually
cmap_jet = plt.get_cmap('jet')
num_domains = np.max(ds.inferred_domains_cells) + 1

fig, axs = plt.subplots(1, num_domains, figsize=(4*num_domains, 4))
for domain_idx in tqdm(range(num_domains)):
    ax = axs[domain_idx]
    plt.sca(ax)
    plt.axis('off')
    cmap = ListedColormap([cmap_jet(lbl_idx / num_domains) if domain_idx == lbl_idx else "#cccccc" for lbl_idx in range(num_domains)])
    ds.plot_domains(rotate=1, cmap=cmap)
    ax.set_ylim(2600, 1900)
    ax.set_xlim(1600, 2300)
    ax.set_title(f"Tissue Domain {domain_idx + 1}")
plt.tight_layout()
plt.savefig('../figures/ssam_sample1a_tissueDomains.pdf', dpi = 500)

In [ ]:
# Plot merged domains
plt.figure(figsize=[5, 5])
ds.plot_domains(rotate=1)
plt.ylim(2600, 1900)
plt.xlim(1600, 2300)
plt.axis('off')
plt.tight_layout()
plt.savefig('../figures/ssam_sample1a_tissueDomainsMerge.pdf', dpi = 500)
plt.show()